In [1]:
import pandas as pd
import numpy as np
#Read data from a CSV file into the variable df
df = pd.read_csv('..\data\movies_metadata.csv')

C:\Users\kasid\AppData\Local\Temp\ipykernel_30788\1535337570.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('..\data\movies_metadata.csv')


In [2]:
#Display all the columns in the dataframe
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
#Select only the columns you want to use:
df = df[['title','genres', 'release_date', 'runtime','vote_average','vote_count']]

#Display the data in tabular format
df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [6]:
# Convert release date to pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Create a column 'year' to only include the year information
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [7]:
#Check the data types of the dataframe
df.info

<bound method DataFrame.info of                              title  \
0                        Toy Story   
1                          Jumanji   
2                 Grumpier Old Men   
3                Waiting to Exhale   
4      Father of the Bride Part II   
...                            ...   
45461                       Subdue   
45462          Century of Birthing   
45463                     Betrayal   
45464             Satan Triumphant   
45465                     Queerama   

                                                  genres release_date  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   1995-10-30   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   1995-12-15   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   1995-12-22   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   1995-12-22   
4                         [{'id': 35, 'name': 'Comedy'}]   1995-02-10   
...                                                  ...          ...   


In [8]:
# Define a function to convert NaT to 0 and year data to integers
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

# Apply the convert_int function to the 'year' column
df['year'] = df['year'].apply(convert_int)

In [9]:
# Drop the 'release_date' column
df = df.drop('release_date', axis=1)

# Display the data in tabular format
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [10]:
#View the structure of how genre data is stored for the first movie:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
from ast import literal_eval

# Test the use of literal_eval by assuming a string data
# Create a stringified list and check its type
a = "[1,2,3]"
print(type(a))

# Use literal_eval and check the type
b = literal_eval(a)
print(type(b))


<class 'str'>
<class 'list'>


In [12]:
# Convert NaN values to stringified empty lists
df['genres'] = df['genres'].fillna('[]')

# Use literal_eval to convert strings to lists of dictionaries
df['genres'] = df['genres'].apply(literal_eval)

# Convert list of dictionaries to list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# Display the data in tabular format
df.head()


,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [14]:
# Split genres into individual entries
s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)

# Rename the new feature as 'genre'
s.name = 'genre'

# Create a new dataframe named gen_df and remove the original 'genres' column
# Add a new column 'genre' to gen_df
gen_df = df.drop('genres', axis=1).join(s)

#Display the data in tabular format
gen_df.head()

C:\Users\kasid\AppData\Local\Temp\ipykernel_30788\1019456764.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy


In [20]:
def build_chart(gen_df, percentile=0.8):
    # Ask user for preferred genre
    print("Input preferred genre")
    genre = input()
    
    # Ask user for shortest duration
    print("Input shortest duration")
    low_time = int(input())

    # Ask user for longest duration
    print("Input longest duration")
    high_time = int(input())
    
    # Ask user for earliest year
    print("Input earliest year")
    low_year = int(input())
    
    # Ask user for latest year
    print("Input latest year")
    high_year = int(input())
    
    # Create a copy of the dataframe
    movies = gen_df.copy()
    
    # Filter movies based on user input
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    # Calculate the mean vote average
    C = movies['vote_average'].mean()
    # Calculate the minimum number of votes required for a movie to be included
    m = movies['vote_count'].quantile(percentile)
    
    # Filter movies that meet the minimum vote count
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    # Calculate the score for each movie
    q_movies['score'] = q_movies.apply(lambda x:
                                       (x['vote_count']/(x['vote_count']+m) * x['vote_average']) +
                                       (m/(m+x['vote_count']) * C), axis=1)
    
    # Sort movies by score in descending order
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [21]:
#Movie Recommendation List
build_chart(gen_df).head()

Input preferred genre
Adventure
Input shortest duration
50
Input longest duration
180
Input earliest year
1999
Input latest year
2010


,title,runtime,vote_average,vote_count,year,genre,score
15480,Inception,148.0,8.1,14075.0,2010,Adventure,7.991759
5481,Spirited Away,125.0,8.3,3968.0,2001,Adventure,7.927076
4863,The Lord of the Rings: The Fellowship of the Ring,178.0,8.0,8892.0,2001,Adventure,7.840190
5814,The Lord of the Rings: The Two Towers,179.0,8.0,7641.0,2002,Adventure,7.816153
3456,Gladiator,155.0,7.9,5566.0,2000,Adventure,7.665867


In [23]:
#convert the DataFrame to a CSV file, use the to_csv
df.to_csv('..\data\metadata_clean.csv', index=False)